In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import os
import functools
from tqdm.auto import tqdm

In [2]:
dataset_path = os.path.join(os.path.abspath(os.path.curdir),'example_data')
trainFiles = os.listdir(dataset_path)

In [3]:
categories=[]  # 0:정상, 1:B / 2:C / 3:F
train_files = []
for trainFile in trainFiles:
    if '.csv' in trainFile:
        categorie=trainFile.split("_")[0]
        train_files.append(trainFile)
        if categorie == "A":
            categories.append("0")
        elif categorie == "B":
            categories.append("1")
        elif categorie == "C":
            categories.append("2")
        elif categorie == "F":
            categories.append("3")

In [4]:
import numpy as np
from PIL import Image
from scipy import optimize

In [5]:
def calc_R(xc,yc):
    """ calculate the distance of each data points from the center (xc, yc) """
    return np.sqrt((x-xc)**2 + (y-yc)**2)

def f_2(c):
    Ri = calc_R(*c)
    return Ri - Ri.mean()

In [15]:
from mpl_toolkits.mplot3d import Axes3D
def d3_now_image(train_files):
    save_dir = os.path.join(os.path.abspath(os.path.curdir) , '3D_image')
    for file_name in train_files:
        if '.csv' in file_name:
            xs,ys,zs = get_3d_coord_data(file_name)
            
            fig = plt.figure(figsize=(12, 9))
            ax = fig.add_subplot(111, projection='3d')
            ax.scatter(xs, ys, zs, c=zs, cmap="inferno", s=5, alpha=0.5)
            fig.savefig(os.path.join(save_dir,file_name+'.png'), bbox_inches = 'tight')

In [19]:
def get_3d_coord_data(file_name):
    df = pd.read_csv(os.path.join(dataset_path,file_name), header=None, index_col=None, names=None)

    #위에 2줄 아래 3줄 제거 -> 결측치 , 순서
    df = df[2:-3]

    #왼쪽 1열 우측 2열 제거 -> 결측치
    df = df.drop(df.columns[0], axis = 1)
    for i in range(0, 2):
        df = df.drop(df.columns[-1], axis = 1)

    row,col = df.shape

    views = df.assign(g=df.index % 3)
    xs = views.query('g==0')
    ys = views.query('g==1')
    zs = views.query('g==2')[:-1]
    
    return xs,ys,zs

In [22]:
from matplotlib import animation 
def d3_now_gif(train_files):
    save_dir = os.path.join(os.path.abspath(os.path.curdir) , 'gif_image')
    for file_name in tqdm(train_files):
        if '.csv' in file_name:
            xs,ys,zs = get_3d_coord_data(file_name)
            fig, axs = plt.subplots(ncols=1, figsize=(10, 5), subplot_kw={"projection":"3d"})

            fontlabel = {"fontsize":"large", "color":"gray", "fontweight":"bold"}

            def init():
                axs.set_xlabel("X", fontdict=fontlabel, labelpad=16)
                axs.set_ylabel("Y", fontdict=fontlabel, labelpad=16)
                axs.set_title("Z", fontdict=fontlabel)

                axs.scatter(xs, ys, zs, c=zs, cmap="inferno", s=5, alpha=0.5)

                return fig,

            def animate(i):
                axs.view_init(elev=30., azim=i)
                return fig,

            # Animate
            anim = animation.FuncAnimation(fig, animate, init_func=init,
                                           frames=360, interval=20, blit=True)
            # Save
            anim.save(os.path.join(save_dir,file_name+'.gif'), fps=30)

In [ ]:
d3_now_image(trainFiles)

In [ ]:
d3_now_gif(trainFiles)

  0%|          | 0/22 [00:00<?, ?it/s]

MovieWriter ffmpeg unavailable; using Pillow instead.


## 기존 코드

In [ ]:
def read_file(pandas):
    
    def calc_R(xc, yc):
        """ calculate the distance of each data points from the center (xc, yc) """
        return np.sqrt((x-xc)**2 + (y-yc)**2)

     # : for output feedback
    def countcalls(fn):
        "decorator function count function calls"

        @functools.wraps(fn)
        def wrapped(*args):
            wrapped.ncalls +=1
            return fn(*args)

        wrapped.ncalls = 0
        return wrapped

    @countcalls
    def f_2(c):
        Ri = calc_R(*c)
        return Ri - Ri.mean()
    
    #첫번째 열 , 마지막 3열 제거
    pandas = pandas.drop(pandas.columns[0], axis = 1)
    for i in range(0, 3):
        pandas = pandas.drop(pandas.columns[-1], axis = 1)

    #각 스캔 파일의 사이즈 확인
    # (7002, 649)
    e, f = pandas.shape

    #각 스캔 파일 결측치 보충을 위한 선형보간법 사용  
    for i in range(0,e):
        for j in range(1,f):
            if pandas.at[i,j]  < 2  :
                pandas.at[i,j] = 0
            elif pandas.at[i,j]  > 7:
                pandas.at[i,j] = 0
                
    # 0을 np.NaN으로 교체후 선형 보간
    pandas = pandas.replace(0, np.NaN)
    pandas = pandas.interpolate(method='linear', axis = 1)            

    # 4,260개 행 데이터 추출(284*5*3)
#     pandas = pandas.loc[e-4662:e-402,:] #2340	 6600 기준이 뭔지 모르겠음

    pac=[]

    for i in range(3,len(pandas)+1,3):
        # 1번째 행 인덱스만 추출
        if i%3==0:
            pac.append(i-1)
        #추출한 인덱스 가지는 df 생성
    pandas = pandas.iloc[pac,:]
    
    # 결측치 교체 후 선형보간
    pandas = pandas.replace(0, np.NaN)
    pandas = pandas.interpolate(method='linear', axis = 1)


    #데이터 이미지화
    pandas_orig = []

    for i in range(0,len(pandas)):
        method_2  = "Fitting Circle"

        # Coordinates of the 2D points
        x = np.linspace(-10.2,10.5,645) #(-10.2 ~ 10.5 까지 645개의 데이터를 일정 간격으로 채움)
        y = pandas.iloc[i,:]            # 행 데이터를 가져옴
        basename = 'arc'

        x_m = np.mean(x)
        y_m = np.mean(y)

         # 최소 제곱법 : 근사적으로 구하려는 해와 실제 해의 오차의 제곱의 합이 최소가 되는 해를 구하는 방법이다.
         # 근사적으로 구하려는 해중 오차가 제일 작은 해를 구함 == center_2
        center_estimate = x_m, y_m
        center_2, _ = optimize.leastsq(f_2, center_estimate)

        xc_2, yc_2 = center_2 #구한 해의 원점

        R=np.sqrt((xc_2-x)**2+(yc_2-y)**2) # 구한 해와 오차를 계산
        pandas_orig.append(R-np.min(R))    # 오차중 최소 값과 차이를 추가
        
    pandas_orig = np.array(pandas_orig)    # 배열로 리턴
    
    return pandas_orig

In [ ]:
import os
for idx,train_file in enumerate(trainFiles):
    if '.csv' in train_file:
        df = pd.read_csv(os.path.join(dataset_path,train_file), header=None, index_col=None, names=None)
        pandas_orig = read_file(df)
        plt.figure(figsize = (5,10), frameon=False)
        plt.imshow(pandas_orig, vmin = 0, vmax = 0.5, cmap='gist_yarg')
        #plt.colorbar(shrink=0.7) # colorbar의 길이를 조정
        plt.axis('off')
        plt.savefig(os.path.join(dataset_path,'picture/'+train_file+'.png'))
        print(idx)